In [1]:
import pandas as pd
import pymysql
import numpy as np


conn = pymysql.connect(host = 'localhost' , user = 'root', password = 'Sheon@915' , db = 'university')
curs = conn.cursor(pymysql.cursors.DictCursor)

drop_sql = """drop table if exists db_score_3_labels"""
curs.execute(drop_sql)
conn.commit()

sql = """
    create table db_score_3_labels(
        sno int primary key,
        homework float,
        discussion int,
        midterm float,
        grade char(1)
    )
    """

curs.execute(sql)
conn.commit()

xl_file = 'db_score_3_labels.xlsx'
db_score_3_labels = pd.read_excel(xl_file)


xl_file_rows = []
for i in db_score_3_labels.values:
    xl_file_rows.append(tuple(i))
    
sql2 = """insert into db_score_3_labels(sno, homework, discussion, midterm, grade) values(%s, %s, %s, %s, %s)"""
curs.executemany(sql2, xl_file_rows)
conn.commit()

sql3 = """select * from db_score_3_labels"""
curs.execute(sql3)
data = curs.fetchall()
display(data)

curs.close()
conn.close()


[{'sno': 1, 'homework': 19.37, 'discussion': 0, 'midterm': 35.0, 'grade': 'A'},
 {'sno': 2, 'homework': 20.0, 'discussion': 0, 'midterm': 34.12, 'grade': 'A'},
 {'sno': 3, 'homework': 19.37, 'discussion': 0, 'midterm': 34.3, 'grade': 'A'},
 {'sno': 4, 'homework': 18.12, 'discussion': 2, 'midterm': 31.5, 'grade': 'A'},
 {'sno': 5, 'homework': 20.0, 'discussion': 2, 'midterm': 33.25, 'grade': 'A'},
 {'sno': 6, 'homework': 19.37, 'discussion': 0, 'midterm': 24.5, 'grade': 'A'},
 {'sno': 7, 'homework': 19.37, 'discussion': 2, 'midterm': 23.8, 'grade': 'A'},
 {'sno': 8,
  'homework': 16.87,
  'discussion': 0,
  'midterm': 28.87,
  'grade': 'A'},
 {'sno': 9,
  'homework': 19.37,
  'discussion': 0,
  'midterm': 29.75,
  'grade': 'A'},
 {'sno': 10, 'homework': 20.0, 'discussion': 0, 'midterm': 31.5, 'grade': 'A'},
 {'sno': 11,
  'homework': 19.37,
  'discussion': 0,
  'midterm': 29.57,
  'grade': 'A'},
 {'sno': 12,
  'homework': 19.37,
  'discussion': 0,
  'midterm': 31.5,
  'grade': 'A'},
 {'

In [2]:
def performance_eval(y_test,y_predict):#첫번째 매개변수 : original data, 두번째 매개변수 : predicted data
    tp, tn, fp, fn = 0, 0, 0, 0
    for y,yp in zip(y_test, y_predict):
        if y == 1 and yp == 1:
            tp += 1 #true positive
        elif y == 1 and yp == -1:
            fn += 1 #false negative
        elif y == -1 and yp == -1:
            tn += 1 #true negative
        else:
            fp += 1 #false positive
            
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2*precision*recall/(precision+recall)
    
    return accuracy, precision, recall, f1_score
            

def performance_train_test_split(X,y):
    from sklearn.model_selection import train_test_split
    from sklearn import tree

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
    
    print(y_train)
    print(y_test)
    
    dtree = tree.DecisionTreeClassifier()
    
    dtree_model = dtree.fit(X_train, y_train)
    
    y_predict = dtree_model.predict(X_test)
    
    accuracy, precision, recall, f1_score = performance_eval(y_test, y_predict)
    
    print("<train_test_split performance evaluation>")
    print("accuracy=%f" %accuracy)
    print("precision=%f" %precision)
    print("recall=%f" %recall)
    print("f1_score=%f" %f1_score)

def performance_k_fold_cross_validation(X,y):
    from sklearn.model_selection import KFold
    from sklearn import tree
    kf = KFold(n_splits = 10, random_state = 42, shuffle = True)
    
    accuracy = []
    precision = []
    recall = []
    f1_score = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        dtree = tree.DecisionTreeClassifier()
        dtree_model = dtree.fit(X_train, y_train)
        y_predict = dtree_model.predict(X_test)
        acc, prec, rec, f1 = performance_eval(y_test, y_predict)
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        f1_score.append(f1)
        
    print("<k_fold_cross_validation performance evaluation>")
    print("accuracy array =", accuracy)
    print("precision array =",precision)
    print("recall array =",recall)
    print("f1_score array =",f1_score)
    
    import statistics
    print("average_accuracy =", statistics.mean(accuracy))
    print("average_precision =", statistics.mean(precision))
    print("average_recall =", statistics.mean(recall))
    print("average_f1_score =", statistics.mean(f1_score))

In [3]:
X = [ (t['sno'], t['homework'], t['discussion'], t['midterm'])  for t in data ]
X = np.array(X)

y = [ 1 if (t['grade'] == 'B') else -1 for t in data ]
y = np.array(y)

performance_train_test_split(X,y)
print('\n')
performance_k_fold_cross_validation(X, y)

[-1 -1  1  1 -1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1
 -1  1 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1  1 -1  1 -1 -1  1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1]
[ 1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1
  1 -1 -1 -1  1 -1 -1]
<train_test_split performance evaluation>
accuracy=0.967742
precision=1.000000
recall=0.888889
f1_score=0.941176


<k_fold_cross_validation performance evaluation>
accuracy array = [1.0, 1.0, 0.8888888888888888, 1.0, 1.0, 1.0, 0.8888888888888888, 1.0, 1.0, 1.0]
precision array = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0]
recall array = [1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
f1_score array = [1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 0.9090909090909091, 1.0, 1.0, 1.0]
average_accuracy = 0.9777777777777777
average_precision = 0.9833333333333334
average_recall = 0.9666666666666667
average_f1_score = 0.9709090909090909


In [4]:
def mc_accuracy_eval(y1_test,y1_predict,y2_test, y2_predict, y3_test, y3_predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    for y,yp in zip(y1_test, y1_predict):
        if y == 1 and yp == 1:
            tp += 1 #true positive
        elif y == -1 and yp == -1:
            tn += 1 #true negative
        elif y == 1 and yp == -1:
            fp += 1 #false positive
        else:
            fn += 1 #false negative
    for y,yp in zip(y2_test, y2_predict):
        if y == 1 and yp == 1:
            tp += 1 #true positive
        elif y == -1 and yp == -1:
            tn += 1 #true negative
        elif y == 1 and yp == -1:
            fp += 1 #false positive
        else:
            fn += 1 #false negative
    for y,yp in zip(y3_test, y3_predict):
        if y == 1 and yp == 1:
            tp += 1 #true positive
        elif y == -1 and yp == -1:
            tn += 1 #true negative
        elif y == 1 and yp == -1:
            fp += 1 #false positive
        else:
            fn += 1 #false negative
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

def mc_others_eval(y_test,y_predict):#첫번째 매개변수 : original data, 두번째 매개변수 : predicted data
    tp, tn, fp, fn = 0, 0, 0, 0
    for y,yp in zip(y_test, y_predict):
        if y == 1 and yp == 1:
            tp += 1 #true positive
        elif y == 1 and yp == -1:
            fn += 1 #false negative
        elif y == -1 and yp == -1:
            tn += 1 #true negative
        else:
            fp += 1 #false positive
            
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2*precision*recall/(precision+recall)
    
    return precision, recall, f1_score

def mc_performance_train_test_split(X,y1, y2, y3):
    from sklearn.model_selection import train_test_split
    from sklearn import tree

    X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size = 0.33, random_state = 42)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size = 0.33, random_state = 42)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X, y3, test_size = 0.33, random_state = 42)
    
    dtree = tree.DecisionTreeClassifier()
    
    dtree_model = dtree.fit(X1_train, y1_train)
    dtree_model2 = dtree.fit(X2_train, y2_train)
    dtree_model3 = dtree.fit(X3_train, y3_train)
    
    y1_predict = dtree_model.predict(X1_test)
    y2_predict = dtree_model2.predict(X2_test)
    y3_predict = dtree_model3.predict(X3_test)
    
    accuracy = mc_accuracy_eval(y1_test, y1_predict, y2_test, y2_predict, y3_test, y3_predict)
  
    print("<train_test_split performance evaluation>")
    print("accuracy=%f" %accuracy)
    
    precision, recall, f1_score = mc_others_eval(y1_test, y1_predict)
    
    print("precision(A)=%f" %precision)
    print("recall(A)=%f" %recall)
    print("f1_score(A)=%f" %f1_score)
    
    precision, recall, f1_score = mc_others_eval(y2_test, y2_predict)
    
    print("precision(B)=%f" %precision)
    print("recall(B)=%f" %recall)
    print("f1_score(B)=%f" %f1_score)
    
    precision, recall, f1_score = mc_others_eval(y3_test, y3_predict)
    
    print("precision(C)=%f" %precision)
    print("recall(C)=%f" %recall)
    print("f1_score(C)=%f" %f1_score)

def mc_performance_k_fold_cross_validation(X,y1, y2, y3):
    from sklearn.model_selection import KFold
    from sklearn import tree
    kf = KFold(n_splits = 4, random_state = 42, shuffle = True)
    
    accuracy = []
    precision = [[],[],[]]
    recall = [[],[],[]]
    f1_score = [[],[],[]]
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y1_train, y1_test = y1[train_index], y1[test_index]
        y2_train, y2_test = y2[train_index], y2[test_index]
        y3_train, y3_test = y3[train_index], y3[test_index]
        
        dtree = tree.DecisionTreeClassifier()
        dtree_model = dtree.fit(X_train, y1_train)
        dtree_model2 = dtree.fit(X_train, y2_train)
        dtree_model3 = dtree.fit(X_train, y3_train)
    
        y1_predict = dtree_model.predict(X_test)
        y2_predict = dtree_model2.predict(X_test)
        y3_predict = dtree_model3.predict(X_test)
        
        acc= mc_accuracy_eval(y1_test, y1_predict, y2_test, y2_predict, y3_test, y3_predict)
        
        accuracy.append(acc)
        
        prec, rec, f1  = mc_others_eval(y1_test, y1_predict)
        precision[0].append(prec)
        recall[0].append(rec)
        f1_score[0].append(f1)
        
        prec, rec, f1  = mc_others_eval(y2_test, y2_predict)
        precision[1].append(prec)
        recall[1].append(rec)
        f1_score[1].append(f1)
        
        prec, rec, f1  = mc_others_eval(y3_test, y3_predict)
        precision[2].append(prec)
        recall[2].append(rec)
        f1_score[2].append(f1)
        
    print("<k_fold_cross_validation performance evaluation>")
    print("accuracy array =", accuracy)
    
    print("precision(A) array =",precision[0])
    print("recall(A) array =",recall[0])
    print("f1_score(A) array =",f1_score[0])
    
    print("precision(B) array =",precision[1])
    print("recall(B) array =",recall[1])
    print("f1_score(B) array =",f1_score[1])
    
    print("precision(C) array =",precision[2])
    print("recall(C) array =",recall[2])
    print("f1_score(C) array =",f1_score[2])
    
    import statistics
    print("average_accuracy =", statistics.mean(accuracy))
    
    print("average_precision(A) =", statistics.mean(precision[0]))
    print("average_recall(A) =", statistics.mean(recall[0]))
    print("average_f1_score(A) =", statistics.mean(f1_score[0]))
    
    print("average_precision(B) =", statistics.mean(precision[1]))
    print("average_recall(B) =", statistics.mean(recall[1]))
    print("average_f1_score(B) =", statistics.mean(f1_score[1]))
    
    print("average_precision(C) =", statistics.mean(precision[2]))
    print("average_recall(C) =", statistics.mean(recall[2]))
    print("average_f1_score(C) =", statistics.mean(f1_score[2]))


In [5]:
X = [ (t['sno'], t['homework'], t['discussion'], t['midterm'])  for t in data ]
X = np.array(X)

y1 = [ 1 if (t['grade'] == 'A') else -1 for t in data ]
y1 = np.array(y1)

y2 = [ 1 if (t['grade'] == 'B') else -1 for t in data ]
y2 = np.array(y1)

y3 = [ 1 if (t['grade'] == 'C') else -1 for t in data ]
y3 = np.array(y1)


mc_performance_train_test_split(X,y1, y2, y3)
print('\n')
mc_performance_k_fold_cross_validation(X, y1, y2, y3)

<train_test_split performance evaluation>
accuracy=0.967742
precision(A)=0.916667
recall(A)=1.000000
f1_score(A)=0.956522
precision(B)=0.916667
recall(B)=1.000000
f1_score(B)=0.956522
precision(C)=0.916667
recall(C)=1.000000
f1_score(C)=0.956522


<k_fold_cross_validation performance evaluation>
accuracy array = [1.0, 0.9565217391304348, 1.0, 1.0]
precision(A) array = [1.0, 0.9, 1.0, 1.0]
recall(A) array = [1.0, 1.0, 1.0, 1.0]
f1_score(A) array = [1.0, 0.9473684210526316, 1.0, 1.0]
precision(B) array = [1.0, 0.9, 1.0, 1.0]
recall(B) array = [1.0, 1.0, 1.0, 1.0]
f1_score(B) array = [1.0, 0.9473684210526316, 1.0, 1.0]
precision(C) array = [1.0, 0.9, 1.0, 1.0]
recall(C) array = [1.0, 1.0, 1.0, 1.0]
f1_score(C) array = [1.0, 0.9473684210526316, 1.0, 1.0]
average_accuracy = 0.9891304347826086
average_precision(A) = 0.975
average_recall(A) = 1.0
average_f1_score(A) = 0.986842105263158
average_precision(B) = 0.975
average_recall(B) = 1.0
average_f1_score(B) = 0.986842105263158
average_precisi